# MuSiQue multi-hop baseline

In [ ]:
#|default_exp musique.nhop

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export

import json
from typing import Callable

import pandas as pd
from tqdm.auto import tqdm
from pydantic import BaseModel

from bellek.jerx.reward.llm import QuestionAnsweringResult
from bellek.musique.eval import calculate_metrics, compare_answers

tqdm.pandas()

In [ ]:
#|export

def make_docs(example):
    ps = example["paragraphs"]
    for p in ps:
        idx = p["idx"]
        title = p["title"]
        body = p["paragraph_text"]
        is_supporting = p["is_supporting"]
        text = f"# {title}\n{body}"
        yield dict(
            text=text,
            is_supporting=is_supporting,
            parent_id=example["id"],
            idx=idx,
        )

In [ ]:
#|export

class Result(BaseModel):
    answer: str
    hops: list[dict]

class BaselineMultiHop:
    def __init__(self, qa_func, retrieval_func):
        self.qa_func = qa_func
        self.retrieval_func = retrieval_func

    def _call(self, example) -> Result:
        docs = list(make_docs(example))
        
        # First question
        prev_answer = None
        hops = []
        for i in range(len(example["question_decomposition"])):
            question = example["question_decomposition"][i]["question"]
            if prev_answer is not None:
                question = question.replace(f"#{i}", prev_answer)
            docs = self.retrieval_func(docs, question)
            context = "\n".join(doc['text'] for doc in docs)
            result = self.qa_func(context=context, question=question)
            hop = {
                "question": question,
                "context": context,
                "answer": result.answer,
                "reasoning": result.reasoning,
            }
            hops.append(hop)
            prev_answer = result.answer
            if result.answer == "N/A":
                break

        # Second question
        return Result(
            answer=hops[-1]['answer'],
            hops=hops,
        )

    def __call__(self, example, ignore_errors: bool = False) -> Result:
        try:
            output = self._call(example)
        except Exception as exc:
            if ignore_errors:
                id = example['id']
                print(f"Failed to answer the question {id}\n{exc}")
                output = Result(answer="N/A", hops=[dict(error=str(exc))])
            else:
                raise
        return output

In [ ]:
#|export

def benchmark(
    dataf: pd.DataFrame,
    qa_func: Callable,
    retrieval_func: Callable,
    ignore_errors: bool = False,
) -> tuple[pd.DataFrame, dict]:
    pipeline = BaselineMultiHop(qa_func, retrieval_func)

    def process(example):
        output = pipeline(example, ignore_errors=ignore_errors)
        example["predicted_answer"] = output.answer
        example["raw_output"] = output
        return example

    dataf = dataf.progress_apply(process, axis=1)
    dataf = compare_answers(dataf)
    scores = calculate_metrics(dataf)
    scores["fuzzy_match"] = dataf["fuzzy_match"].mean()
    return dataf, scores

In [ ]:
df = pd.read_json('../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)
print(len(df))
df.head()

1252


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__131818_161450,"[{'idx': 0, 'title': 'Maria Carrillo High Scho...",Where is the Voshmgir District located?,"[{'id': 131818, 'question': 'Which state is Vo...",in the north-east of the country south of the ...,"[Caspian Sea, in the north-east of the country...",True,"[Caspian Sea, in the north-east of the country..."
1,2hop__444265_82341,"[{'idx': 0, 'title': 'Ocala, Florida', 'paragr...",In what part of Florida is Tom Denney's birthp...,"[{'id': 444265, 'question': 'Tom Denney >> pla...",in Northern Florida,"[in Northern Florida, Northern Florida]",True,"[in Northern Florida, Northern Florida]"
2,2hop__711946_269414,"[{'idx': 0, 'title': 'Wild Thing (Tone Lōc son...",What record label is the performer who release...,"[{'id': 711946, 'question': 'All Your Faded Th...",Kill Rock Stars,[Kill Rock Stars],True,[Kill Rock Stars]
3,2hop__311931_417706,"[{'idx': 0, 'title': 'The Main Attraction (alb...",What record label does the performer of Emotio...,"[{'id': 311931, 'question': 'Emotional Rain >>...",Attic Records,"[Attic, Attic Records]",True,"[Attic, Attic Records]"
4,2hop__809785_606637,"[{'idx': 0, 'title': 'The Main Attraction (alb...",What record label does the performer of Advent...,"[{'id': 809785, 'question': 'Adventures in You...",Secret City Records,[Secret City Records],True,[Secret City Records]


In [ ]:
from bellek.qa.llm import make_question_answer_func

qa_func = make_question_answer_func()
retrieval_func = lambda docs, query: docs
pipeline = BaselineMultiHop(qa_func, retrieval_func)

In [ ]:
from bellek.utils import jprint

i = 1
example = df.iloc[i].to_dict()
output = pipeline(example).dict()
print("Question:", example['question'])
print("Reference answer:", example['answer'])
print("Predicted answer:", output['answer'])
for hop in output['hops']:
    jprint(hop)

Question: In what part of Florida is Tom Denney's birthplace located?
Reference answer: in Northern Florida
Predicted answer: Northern Florida
{
  "question": "Tom Denney >> place of birth",
  "context": "# Ocala, Florida\nOcala (/ o\u028a \u02c8k\u00e6l\u0259 / oh - KAL - \u0259) is a city located in Northern Florida. As of the 2013 census, its population, estimated by the United States Census Bureau, was 57,468, making it the 45th most populated city in Florida.\n# Jasmine Estates, Florida\nJasmine Estates is a census-designated place (CDP) in Pasco County, Florida, United States. The population was 18,989 at the 2010 census.\n# Ridgecrest, Florida\nRidgecrest is a census-designated place (CDP) in Pinellas County, Florida, United States. The population was 2,558 at the 2010 census.\n# Villas, Florida\nVillas is a census designated place (CDP) in Lee County, Florida, United States. The population was 11,346 at the 2000 census. It is part of the Cape Coral-Fort Myers, Florida Metropoli

In [ ]:
mdf, scores = benchmark(df.sample(2), qa_func, retrieval_func)
print(scores)
mdf

  0%|          | 0/2 [00:00<?, ?it/s]

{'exact_match': 0.5, 'f1': 0.5, 'fuzzy_match': 0.5}


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers,predicted_answer,raw_output,exact_match,fuzzy_match
511,2hop__166820_52835,"[{'idx': 0, 'title': 'Crook & Chase', 'paragra...","Who wrote turn me on, which was performed by t...","[{'id': 166820, 'question': 'Chasing Pirates >...",John D. Loudermilk,[John D. Loudermilk],True,[John D. Loudermilk],John D. Loudermilk,answer='John D. Loudermilk' hops=[{'question':...,True,True
731,2hop__147260_623159,"[{'idx': 0, 'title': 'History of Singapore', '...",What territorial entity does the town where Jo...,"[{'id': 147260, 'question': 'Where did John th...",Yaroslavl Oblast,[Yaroslavl Oblast],True,[Yaroslavl Oblast],Derge Agen,answer='Derge Agen' hops=[{'question': 'Where ...,False,False


In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()